In [ ]:
!apt update
!apt install -y cmake
!pip install dlib

In [11]:
!git clone https://github.com/indrajitkurmi/PyCLFRTryColab

Cloning into 'PyCLFRTryColab'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 2331 (delta 45), reused 6 (delta 4), pack-reused 2267
Receiving objects: 100% (2331/2331), 255.12 MiB | 27.71 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Checking out files: 100% (2241/2241), done.


In [14]:
%ls
%cd PyCLFRTryColab/glesLFR/include

PyCLFRTryColab/  sample_data/
/content/PyCLFRTryColab/glesLFR/src


In [ ]:
!git clone https://github.com/assimp/assimp
%cd PyCLFRTryColab/glesLFR/include/assimp
!make

In [ ]:
%cd PyCLFRTryColab/glesLFR/include
!git clone https://github.com/glfw/glfw
%cd PyCLFRTryColab/glesLFR/include/glfw
!cmake

In [15]:
!python TestRendererBind_setup_Indrajit.py build_ext --inplace

running build_ext
skipping 'glesLFR_Indrajit_Pyth.cpp' Cython extension (up-to-date)
building 'glesLFR_Indrajit' extension
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c glesLFR_Indrajit_Pyth.cpp -o build/temp.linux-x86_64-3.7/glesLFR_Indrajit_Pyth.o -Wall -Wextra -std=c++17 -ggdb -lpthread -I../include/ -L../lib/ -llibassimp.so.5
glesLFR_Indrajit_Pyth.cpp:630:10: fatal error: numpy/arrayobject.h: No such file or directory
 #include "numpy/arrayobject.h"
          ^~~~~~~~~~~~~~~~~~~~~
compilation terminated.
error: command 'x86_64-linux-gnu-gcc' failed with exit status 1


In [ ]:
print('ProgramStarted')
import glesLFR_Indrajit
import json
import numpy as np
import os
import cv2
print('glesLFR Import')

In [ ]:

def ReadJsonPosesFiles(PyClassObject,PosesFilePath):
        with open(PosesFilePath) as PoseFile:
            PoseFileData = json.load(PoseFile)
            NoofPoses = len(PoseFileData['images'])
            PoseFileImagesData = PoseFileData['images']
            if len(PoseFileData['images']) > 0:
                PyClassObject.Py_allocate(len(PoseFileData['images']))
                for i in range (0,len(PoseFileData['images'])):
                    ImageName = PoseFileImagesData[i]['imagefile']
                    #Convert List of List to Np array
                    PoseMatrix = PoseFileImagesData[i]['M3x4']
                    PoseMatrixNumpyArray = np.array([], dtype=np.double)
                    for k in range(0,len(PoseMatrix)):
                        PoseMatrixNumpyArray = np.append(PoseMatrixNumpyArray, np.asarray(PoseMatrix[k],dtype=np.double))
                    PyClassObject.namespushback(ImageName)
                    PyClassObject.posespushback(np.array(PoseMatrixNumpyArray))
        return NoofPoses
def RenderInitialize(PyClassObject,PosesFilePath,ImageLocation,ObjModelPath,ObjModelImagePath):
        PyClassObject.RenderInitializeP1()
        NofPoses = ReadJsonPosesFiles(PyClassObject, PosesFilePath)
        print(NofPoses)
        PyClassObject.Py_LoadDemModel(ObjModelPath,ObjModelImagePath)
        for i in range (0,NofPoses):
            ImageName = PyClassObject.GetnamesIndex(i)
            print(ImageName)
            LoadImageName = ImageName.replace('.tiff','.png')
            Image = cv2.imread(os.path.join(ImageLocation,LoadImageName),0)
             # height, width, number of channels in image
            height = Image.shape[0]
            width = Image.shape[1]
            if len(Image.shape) == 2 :
                nrComponents = 1
            else :
                nrComponents = Image.shape[2]
            PyClassObject.LoadImageToC_8bit_1ch(Image)
            PyClassObject.Py_GenrateTextureID()
            PyClassObject.Py_BindImageWithTextureID(i,height,width,nrComponents)
        print('Binding Texture Successful')
        PyClassObject.RenderInitializeP2()
        print('Inititalization Finished')

In [ ]:
PosesFilePath = '../data/T20200207F2/thermal_GPS_Corr.json'
ImageLocation = '../data/T20200207F2/thermal_ldr512'
ObjModelPath = '../data/T20200207F2/dem.obj'
ObjModelImagePath = '../data/T20200207F2/dem.png'
FocalLength = 50.815436217896945
PyLFClass = glesLFR_Indrajit.PyLightfieldClass(0)
PyLFClass.Py_setCameraFocalLength(FocalLength)
PyLFClass.Py_setProjectionMatrix()
RenderInitialize(PyLFClass,PosesFilePath,ImageLocation,ObjModelPath,ObjModelImagePath)
ImageReturned1 = PyLFClass.RenderImageOnce('RenderedImage1.png')
cv2.imwrite('Image1InMainApp.png', ImageReturned1)
ImageReturned2 =  PyLFClass.RenderImageOnce('RenderedImage2.png')
cv2.imwrite('Image2InMainApp.png', ImageReturned2)
PyLFClass.TerminateRendererOnceFinished()
#ReadJsonPosesFiles('../data/T20200207F2/thermal_GPS_Corr.json')
print('PY LightFieldClass generated')

#PyLFClass = glesLFR_Indrajit.PyLightfieldClass(0)
#glesLFR_Indrajit.Py_PrintPyLightFieldInstanceInfo(PyLFClass)
#glesLFR_Indrajit.Py_Initiaterender()
#PyLFClass.RenderImageOnce()
#glesLFR_Indrajit.Py_Completerender()

#glesLFR_Indrajit.Py_Initiaterender()
del(PyLFClass)
